In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dspy
from dspy.datasets import DataLoader
from dspy.teleprompt import LabeledFewShot
from dspy.evaluate import Evaluate
from dspy.evaluate.metrics import answer_exact_match
from typing import List, Any


In [ ]:
lm = dspy.LM(model="openai/Qwen/Qwen2-VL-72B-Instruct", api_base="http://localhost:8000/v1", api_key="sk-proj-1234567890")

# adapter = dspy.ChatAdapter()
dspy.settings.configure(lm=lm)

In [ ]:
input_keys = tuple([f"image_{i}" for i in range(1, 3)] + ["question", "options"])
subsets = ['Accounting', 'Agriculture', 'Architecture_and_Engineering', 'Art', 'Art_Theory', 'Basic_Medical_Science', 'Biology', 'Chemistry', 'Clinical_Medicine', 'Computer_Science', 'Design', 'Diagnostics_and_Laboratory_Medicine', 'Economics', 'Electronics', 'Energy_and_Power', 'Finance', 'Geography', 'History', 'Literature', 'Manage', 'Marketing', 'Materials', 'Math', 'Mechanical_Engineering', 'Music', 'Pharmacy', 'Physics', 'Psychology', 'Public_Health', 'Sociology']

devset = []
valset = []
for subset in subsets:
    dataset = DataLoader().from_huggingface("MMMU/MMMU", subset, split=["dev", "validation"], input_keys=input_keys)
    devset.extend(dataset["dev"])
    valset.extend(dataset["validation"])
    

In [ ]:
def count_images(dataset):
    image_counts = {i: 0 for i in range(3)}  # Initialize counts for 0 to 2 images
    for example in dataset:
        count = sum(1 for key in example.inputs().keys() if key.startswith('image_') and example.inputs()[key] is not None)
        if count <= 2:
            image_counts[count] += 1
    return image_counts

max_images = 1
devset = [example for example in devset if sum(1 for key in example.inputs().keys() if key.startswith('image_') and example.inputs()[key] is not None) <= max_images]
valset = [example for example in valset if sum(1 for key in example.inputs().keys() if key.startswith('image_') and example.inputs()[key] is not None) <= max_images]

devset_image_counts = count_images(devset)
valset_image_counts = count_images(valset)

print("Image counts in devset:")
for count, num_examples in devset_image_counts.items():
    print(f"{count} image(s): {num_examples} examples")

print("\nImage counts in valset:")
for count, num_examples in valset_image_counts.items():
    print(f"{count} image(s): {num_examples} examples")


In [ ]:
Image = Any
class MMMUSignature(dspy.Signature):
    """Output a rationale and the answer to a multiple choice question about an image."""

    question: str = dspy.InputField(desc="A question about the image(s)")
    image_1: Image = dspy.InputField(desc="An image of a math problem")
    # image_2: Image = dspy.InputField(desc="An image of a math problem")
    options: List[str] = dspy.InputField(desc="The options to the question")
    answer: str = dspy.OutputField(desc="The answer to the question")

predictor = dspy.ChainOfThought(MMMUSignature, temperature=0.0)
# print(predictor(question="What is the capital of france?"))

In [ ]:
# sample_input=devset[0]
results = []
for sample_input in devset[10:20]:
    x = predictor(**sample_input.inputs())
    results.append(x)
    print(x)

# predictor(**devset[0].inputs())

In [ ]:
lm.inspect_history()

In [ ]:
# raise ValueError("stop here")

In [11]:
evaluate = Evaluate(metric=answer_exact_match, num_threads=300, devset= valset, display_progress=True)
print(evaluate(predictor))

In [ ]:
# print(len(devset))
# print(devset)

optimizer = LabeledFewShot(k=3)
bootstrapped_predictor = optimizer.compile(predictor, trainset=devset, sample=False)
# print(sample_input.inputs())
# print(encode_image(sample_input.inputs()["image_1"]))
# print(sample_input.inpi)
# print(bootstrapped_predictor(**sample_input.inputs()))

# evaluate = Evaluate(metric=answer_exact_match, num_threads=300, devset= valset, display_progress=True)
# print(evaluate(bootstrapped_predictor))

In [ ]:
test_input = dataset["dev"][0]
test_input